In [3]:
import os
import fileinput
from schnetpack.datasets import QM9
import numpy as np
import pandas as pd
from numpy import savetxt

#Load qm9data
qm9data = QM9('./qm9.db', download=True, remove_uncharacterized=True)

#Input range of QM9 to be labelled
print('how many QM9 molecules do you want to label?')
#number_of_molecules = input()
#number_of_molecules = int(number_of_molecules)

how many QM9 molecules do you want to label?


In [15]:
def get_coordinates():
    x = props['_positions'][ :,0]
    y = props['_positions'][ :,1]
    z = props['_positions'][ :,2]
    x = x.numpy()
    y = y.numpy()
    z = z.numpy()
    return x, y, z

def write_coordinates_xyz():    
    x = get_coordinates()[0]
    y = get_coordinates()[1]
    z = get_coordinates()[2] 
    file_xyz = open(name_xyz, mode='w',encoding='utf-8')
    
    number_of_atoms = len(z)
    number_of_atoms = int(number_of_atoms)
    file_xyz.write(str(len(z))+'\n')    
    file_xyz.write('title'+'\n')
    for i in range(number_of_atoms):
        if props['_atomic_numbers'][i] == 1:
            string = ('H' + ' ' +  str(x[i]) + ' ' + str(y[i]) + ' ' + str(z[i]) + '\n')
            file_xyz.write(string)
        if props['_atomic_numbers'][i] == 6:
            string = ('C' + ' ' +  str(x[i]) + ' ' + str(y[i]) + ' ' + str(z[i]) + '\n')
            file_xyz.write(string)
        if props['_atomic_numbers'][i] == 8:
            string = ('O' + ' ' +  str(x[i]) + ' ' + str(y[i]) + ' ' + str(z[i]) + '\n')
            file_xyz.write(string)
        if props['_atomic_numbers'][i] == 7:
            string = ('N' + ' ' +  str(x[i]) + ' ' + str(y[i]) + ' ' + str(z[i]) + '\n')
            file_xyz.write(string)
        if props['_atomic_numbers'][i] == 9:
            string = ('F' + ' ' +  str(x[i]) + ' ' + str(y[i]) + ' ' + str(z[i]) + '\n')
            file_xyz.write(string)
    file_xyz.close()

def store_positions():
    countmollines = 0
    array_of_element = []
    for line in fileinput.FileInput(name_mol,inplace=0):
        countmollines = countmollines + 1
        if element in line:
            array_of_element.append([])
            array_of_element[len(array_of_element)-1] = int(countmollines-4)
    return array_of_element



#THe repeating patterns below are actually slightly different, the first deals with appending elements of a vector into a matrix
# Whereas the second appends elements into a vector... so they cannot be reduced much further.. 
def connection_matrix():
    count=0
    connected_positions = []
    array = []
    for j in range(len(array_of_element)):
        connected_positions.append([])
        array.append([])
        array[j] = str(array_of_element[j])
    for line in fileinput.FileInput(name_mol,inplace=0):
        count = count + 1
        if count >= number_atoms + 5:
            line = line.replace(line,line[:7])
            for j in range(len(array_of_element)):
                if ' ' + str(array_of_element[j]) + ' ' in line and 'RAD' not in line:
                    connected_positions[j].append([])
                    if str(line[1]+line[2]+line[3]) == ' '+array[j]+' ' or str(line[0]+line[1]+line[2]+line[3]) == ' '+array[j]+' ':
                        connected = str(line[4:6])
                        connected = int(connected)
                        connected_positions[j][len(connected_positions[j])-1]=connected
                    else:
                        connected = str(line[1]+line[2]+line[3])
                        connected = int(connected)
                        connected_positions[j][len(connected_positions[j])-1]=connected
#    print(connected_positions)
    return connected_positions

def neighboring_connections():
    carbon_neighbor_connections = []
    countmollines = 0
    for line in fileinput.FileInput(name_mol,inplace=0):
        countmollines = countmollines + 1
        if countmollines >= number_atoms + 5:
            line = line.replace(line,line[:7])
            if ' ' + str(carbon_position) + ' ' in line and 'RAD' not in line:
                carbon_neighbor_connections.append([])
                if str(line[1]+line[2]+line[3]) == ' '+str(carbon_position)+' ' or str(line[0]+line[1]+line[2]+line[3]) == ' '+str(carbon_position)+' ':
                    connected = str(line[4:6])
                    connected = int(connected)
                    carbon_neighbor_connections[len(carbon_neighbor_connections)-1]=connected
                else:
                    connected = str(line[1]+line[2]+line[3])
                    connected = int(connected)
                    carbon_neighbor_connections[len(carbon_neighbor_connections)-1]=connected
    return carbon_neighbor_connections

def label_hydrogen():
    countC2 = np.zeros((len(connected_positions)))
    countH2 = np.zeros((len(connected_positions)))
    countO2 = np.zeros((len(connected_positions)))
    countN2 = np.zeros((len(connected_positions)))
    label = 0
    carbon_neighbor_connections = neighboring_connections()
    for i in range(len(carbon_neighbor_connections)):
        countxyzlines = 0
        for line2 in fileinput.FileInput(name_xyz,inplace=0):
            countxyzlines = countxyzlines + 1
            if countxyzlines == carbon_neighbor_connections[i] + 2:
                if 'C ' in line2:
                    countC2[j] = countC2[j] + 1
                if 'H ' in line2:
                    countH2[j] = countH2[j] + 1
                if 'N ' in line2:
                    countN2[j] = countN2[j] + 1
                if 'O ' in line2:
                    countO2[j] = countO2[j] + 1

    #count total number of atoms for cleaner results

    total=countH2[j]+countN2[j]+countO2[j]+countC2[j]
    if group == 1.0:                                
        if total == 4:    
            if countH2[j] == 4:
                #this is methane, no cluster
                label = 1
            elif countH2[j] == 3 and countC2[j] == 1:
                #these are all the methyl groups
                label = 2
            #methyl-O
            elif countH2[j] == 3 and countO2[j] == 1:
                label = 3
            #methyl-N
            elif countH2[j] == 3 and countN2[j] == 1:
                label = 4
            #this is methylene
            elif countH2[j] == 2 and countC2[j] == 2:
                label = 5
            #methylene-N
            elif countH2[j]==2 and countC2[j]==1 and countN2[j]==1:
                label = 6
            elif countH2[j] == 2 and countC2[j] == 1 and countO2[j]==1:
                label = 7
            elif countH2[j] == 2 and countO2[j] == 2:
                label = 8
            #methines
            elif countH2[j]==1 and countC2[j]==3:
                label=9
            elif countH2[j]==1 and countC2[j]==2 and countN2[j]==1:
                label=10
            elif countH2[j]==1 and countC2[j]==2 and countO2[j]==1:
                label=11
            #outer acetal
            elif countH2[j]==1 and countC2[j]==1 and countO2[j]==2:
                label=12 
 
        elif total == 3:
            #trigonal planar carbons
            #arromatic
            if countH2[j]==1 and countC2[j]==2:
                label=13
            elif countH2[j]==1 and countC2[j]==1 and countO2[j]==1:
                label=14
            elif countH2[j]==1 and countC2[j]==1 and countN2[j]==1:
                label=15
            elif countH2[j]==1 and countO2[j]==2:
                label=16
            elif countH2[j]==1 and countN2[j]==2:
                label=17
                print('17!!!!!!!')
            elif countH2[j]==1 and countN2[j]==1 and countO2[j]==1:
                label=18
                print('18!!!!!!!!')
            elif countH2[j]==2 and countO2[j]==1:
                label=19
                print('19!!!!!!!!')
                #formaldehyde
        elif total == 2:
            if countH2[j]==1 and countC2[j]==1:
                label=20
                print('20!!!!!!!!!')
            elif countH2[j]==1 and countN2[j]==1:
                label=21
                print('21!!!!!!!!!')
    if group == 2.0: 
        if total == 3:
            #trigonal planar nitrogens
            #first one is ammonia, no cluster
            #ammonia            
            if countH2[j]==3:
                label=22
                print('22!!!!!!!!')
            #primary amine
            elif countH2[j]==2 and countC2[j]==1:
                label=23
                print('23!!!!!!!!')
            #secondary amine
            elif countH2[j]==1 and countC2[j]==2:
                label=24
                print('24!!!!!!!!')
            elif countH2[j]==1 and countN2[j]==1 and countC2[j]==1:
                label=25
                print('25!!!!!!!!')
            elif countH2[j]==1 and countN2[j]==2:
                label=26
                print('26!!!!!!!!')
        elif total==2:
            if countH2[j]==1 and countC2[j]==1:
                label=27
                print('27!!!!!!!!')

    if group==3.0:
        if total==2:
            #alcohols
            if countH2[j]==1 and countC2[j]==1:
                label=28
                print('28!!!!!!!!')
                positions = connected_positions[i]
                carbon_neighbor = min(positions)

                index = positions.index(carbon_neighbor)
                positions.pop(index)

                nn = neighboring_connections()
                total3,countC3,countH3,countO3,countN3 = count_nn(nn,name_xyz)
                
                if total3 == 4:
                    label = 29

                if total3 == 3:
                    label = 30
            print(str(label)+'!!!!!!')

            if countH2[j] == 2:
                label=31
                print('31!!!!!!!!')
            if countH2[j]==1 and countN2[j]==1:
                label=32
                print('32!!!!!!!!')
    if label == 0:
        print('hereZERO!!!!')
    return label

def categorize_hydrogens():
    ##Initialize important data holders 
    countC = np.zeros((len(connected_positions)))
    countH = np.zeros((len(connected_positions)))
    countO = np.zeros((len(connected_positions)))
    countN = np.zeros((len(connected_positions)))
    for i in range(len(connected_position)):
        countxyzlines=0
        for line2 in fileinput.FileInput(name_xyz,inplace=0):
            countxyzlines = countxyzlines+1
            if countxyzlines == connected_positions[j][i]+2:
                if 'C ' in line2:
                    countC[j] = countC[j]+1
                if 'H ' in line2:
                    countH[j] = countH[j]+1
                if 'N ' in line2:
                    countN[j] = countN[j]+1
                if 'O ' in line2:
                    countO[j] = countO[j]+1

#label between alcohol/carbonyl/ether
    if countC[j] == 1:
        group = 1.0
    elif countO[j] == 1:
        group = 3.0
    elif countN[j] == 1:
        group = 2.0
#            else:
#                label[j] = 0
    return group

def count_nn(nn,name_xyz):
    countC = 0
    countH = 0
    countO = 0
    countN = 0
    for j in range(len(nn)):
        target = nn[j]
        countxyzlines = 0
        for line2 in fileinput.FileInput(name_xyz,inplace=0):
            countxyzlines = countxyzlines+1
            if countxyzlines == nn[j]+2:
                if 'C ' in line2:
                    countC = countC+1
                if 'H ' in line2:
                    countH = countH+1
                if 'N ' in line2:
                    countN = countN+1
                if 'O ' in line2:
                    countO = countO+1
    total = countC + countH + countO + countN
    
    return total, countC, countH, countO, countN



# Label Oxygens and Hydrogens

In [16]:
datah = np.zeros((1,2))
datao = np.zeros((1,2))
# Store an array of all oxygen positions in each molecules
for idx in range(5000):
    at, props = qm9data.get_properties(idx)
    
    print(idx)
    name_xyz = '../../../../data/label/5000/' + str(idx) + '.xyz'
    name_mol = '../../../../data/label/5000/' + str(idx) + '.mol'
    
    #write coordinates xyz file
    write_coordinates_xyz()
    
    
    #convert xyz file to mol file using obabel
    os.system('obabel ' + name_xyz + ' -O ' + name_mol)
    

    element = ' H  '
    array_of_element = store_positions()
    
    x = get_coordinates()[0]
    number_atoms = len(x)
    
    # construct the matrix of connection
    if 1 in props['_atomic_numbers']:
        positions = array_of_element
        connected_positions = connection_matrix()
        
        for j in range(len(connected_positions)):
            connected_position = connected_positions[j]
            
            group = categorize_hydrogens() 

            carbon_position = connected_positions[j][0]
        
            label_h = label_hydrogen()
                        
            dt_h = np.array([[label_h, idx]])
            datah = np.vstack((datah,dt_h))
            

                                 
savetxt('../../../../data/label/label-h.csv',datah,delimiter=',')
os.system('rm ../../../../data/label/5000/*mol')
os.system('rm ../../../../data/label/5000/*xyz')
            
savetxt('../../../../data/label/label-o.csv',datao,delimiter=',')
os.system('rm ../../../../data/label/5000/*mol')
os.system('rm ../../../../data/label/5000/*xyz')

            

0
1
22!!!!!!!!
22!!!!!!!!
22!!!!!!!!
2
0!!!!!!
31!!!!!!!!
0!!!!!!
31!!!!!!!!
3
20!!!!!!!!!
20!!!!!!!!!
4
21!!!!!!!!!
5
19!!!!!!!!
19!!!!!!!!
6
7
28!!!!!!!!
28!!!!!!
8
20!!!!!!!!!
9
10
11
23!!!!!!!!
23!!!!!!!!
18!!!!!!!!
12
13
28!!!!!!!!
28!!!!!!
14
15
16
17
18
23!!!!!!!!
23!!!!!!!!
19
23!!!!!!!!
23!!!!!!!!
23!!!!!!!!
23!!!!!!!!
20
21
28!!!!!!!!
28!!!!!!
22
20!!!!!!!!!
20!!!!!!!!!
23
20!!!!!!!!!
24
25
20!!!!!!!!!
26
27
28
29
20!!!!!!!!!
30
31
23!!!!!!!!
23!!!!!!!!
32
28!!!!!!!!
28!!!!!!


UnboundLocalError: local variable 'group' referenced before assignment

In [6]:
savetxt('../../../data/label/50000/label-h.csv',datah,delimiter=',')
os.system('rm ../../../data/label/50000/*mol')
os.system('rm ../../../data/label/50000/*xyz')
            
savetxt('../../../data/label/50000/label-o.csv',datao,delimiter=',')
os.system('rm ../../../data/label/50000/*mol')
os.system('rm ../../../data/label/50000/*xyz')


512

In [ ]:
    element = ' O  '
    array_of_element = store_positions()

    # construct the matrix of connection
    if 8 in props['_atomic_numbers']:
        positions = array_of_element
        connected_positions = connection_matrix()
        
        for j in range(len(connected_positions)):        
            group, countH, countC = categorize_oxygens()

            if countH > 0:
                if connected_positions[j][0] < connected_positions[j][1]:
                    connected_positions[j][0] = connected_positions[j][0]
                else:
                    connected_positions[j][0] = connected_positions[j][1]

            carbon_position = connected_positions[j][0]        
            
            label_o = label_oxygens()
            
            dt_o = np.array([[label_o, idx]])
#            print(dt_o)
            datao = np.vstack((datao,dt_o))





def label_oxygens(): 
    label=0
    countC1 = np.zeros((len(connected_positions)))
    countO1 = np.zeros((len(connected_positions)))
    countH1 = np.zeros((len(connected_positions)))
    countN1 = np.zeros((len(connected_positions)))
    carbon_neighbor_connections = neighboring_connections()
    for i in range(len(carbon_neighbor_connections)):
        countxyzlines = 0
        for line2 in fileinput.FileInput(name_xyz,inplace=0):
            countxyzlines = countxyzlines + 1
            if countxyzlines == carbon_neighbor_connections[i] + 2:
                if 'C ' in line2:
                    countC1[j] = countC1[j] + 1
                if 'H ' in line2:
                    countH1[j] = countH1[j] + 1
                if 'N ' in line2:
                    countN1[j] = countN1[j] + 1
                if 'O ' in line2:
                    countO1[j] = countO1[j] +1
    total=countH1[j]+countN1[j]+countO1[j]+countC1[j]
    if group ==2.0:
        if total==3:
            if countO1[j]==1 and countH1[j]==2:
                #this is formaldehyde
                label=1
            elif countO1[j]==1 and countH1[j]==1 and countC1[j]==1:
                #this is aldehyde
                label=2
            elif countO1[j]==1 and countH1[j]==1 and countN1[j]==1:
                #this is a terminal amide
                label=3
            elif countO1[j]==2 and countH1[j]==1:
                #this is a terminal ester
                label=4
            elif countO1[j]==2 and countC1[j]==1:
                #this is an internal ester
                label=5
            elif countO1[j]==1 and countC1[j]==2:
                #this is ketone
                label=6
            elif countO1[j]==1 and countC1[j]==1 and countN1[j]==1:
                #internal amide
                label=7
            elif countO1[j]==1 and countN1[j]==2:
                #double amide
                label=8
            elif countO1[j]==3:
                label=9
            elif countO1[j]==2 and countN1[j]==1:
                label=33
        if total==2:
            if countO1[j]==1 and countC1[j]==1:
                label==10
            if countO1[j]==1 and countN1[j]==1:
                label=11

    if group == 1.0:
        if total == 4:
            if countO1[j]==1.0 and countH1[j]==3.0:
                #methanol
                label=12
                print(label)
            elif countO1[j]==1 and countH1[j]==2 and countC1[j]==1:
                #primary alcohol
                label=13
            elif countO1[j]==1 and countH1[j]==1 and countC1[j]==2:
                #secondary alcohol
                label=14
            elif countO1[j]==1 and countC1[j]==3:
                #tertiary alcohol
                label=15
            elif countO1[j]==2 and countC1[j]==2:
                label=16
            elif countO1[j]==2 and countC1[j]==1 and countH1[j]==1:
                label=17
            elif countO1[j]==2 and countC1[j]==1 and countN1[j]==1:
                label=18
            elif countO1[j]==2 and countN1[j]==2:
                label=19
            elif countO1[j]==2 and countN1[j]==1 and countH1[j]==1:
                label=20
                
        if total == 3:
        ## NOW ALL THE TRIGONAL PLANARS 
            if countO1[j]==1 and countC1[j]==2:
                label = 21
            elif countO1[j]==1 and countC1[j]==1 and countH1[j]==1:
                label=22
            elif countO1[j]==1 and countN1[j]==1 and countH1[j]==1:
                label=23
            elif countO1[j]==1 and countN1[j]==2:
                label=24
            elif countO1[j]==2 and countH1[j]==1:
                label=25
            elif countO1[j]==2 and countC1[j]==1:
                label=26
                print("acidddss!!!")
            elif countO1[j]==2 and countN1[j]==1:
                label=27
            elif countO1[j]==1 and countN1[j]==1 and countC1[j]==1:
                label=34
        #NOW ALL THE LINEAR
        if total==2:           
            if countO1[j]==1 and countC1[j]==1:
                #alcohol next to triple bond
                label=28
            if countO1[j]==1 and countN1[j]==1:
                label=29
        if total==1:
            #water
            label=30
    if group == 3.0:
        label = 31
    if group == 4.0:
        label =32
    if group==5.0:
        label=35
    return label

def categorize_oxygens():
##Initialize important data holders 
    countC = 0
    countH = 0
    countN = 0
    countO = 0
    group = 0
    label = 0
    connected_position = connected_positions[j]
    for i in range(len(connected_position)):
        countxyzlines=0
        for line2 in fileinput.FileInput(name_xyz,inplace=0):
            countxyzlines = countxyzlines+1
            if countxyzlines == connected_positions[j][i]+2:
                if 'C ' in line2:
                    countC = countC+1
                if 'H ' in line2:
                    countH = countH+1
                if 'N ' in line2:
                    countN = countN+1

#label between alcohol/carbonyl/ether

    if countH > 0 and countC > 0:
        group = 1.0
    elif countN>0:
        group=5.0
    elif countC > 0 and countH == 0:
        if countC == 1:
            group = 2.0
        if countC == 2:
            group = 3.0
    elif countC==0:
        group=4.0

    return group, countH, countC

savetxt('../../../../data/label/label-o.csv',datao,delimiter=',')
os.system('rm ../../../../data/label/5000/*mol')
os.system('rm ../../../../data/label/5000/*xyz')
